In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate, KFold
from surprise.model_selection import train_test_split



sns.set_style("darkgrid")

In [2]:
combined = pd.read_csv('./Sample of ratings with movies.csv')

In [3]:
combined.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
combined.head()

,Cust_Id,Rating,Date,Movie_Id,Year,Title
0,822109,5.0,2005-05-13,1,2003.0,Dinosaur Planet
1,372233,5.0,2005-11-23,1,2003.0,Dinosaur Planet
2,401047,4.0,2005-06-03,1,2003.0,Dinosaur Planet
3,793564,4.0,2004-04-19,1,2003.0,Dinosaur Planet
4,2477242,5.0,2005-01-30,1,2003.0,Dinosaur Planet


In [5]:
combined.isnull().sum()

Cust_Id      0
Rating       0
Date         0
Movie_Id     0
Year        13
Title        0
dtype: int64

In [6]:
combined[combined['Year'].isnull()]

,Cust_Id,Rating,Date,Movie_Id,Year,Title
1252326,1805240,3.0,2004-12-06,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252327,914599,5.0,2005-05-20,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252328,298134,3.0,2002-05-23,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252329,1410134,3.0,2005-05-04,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252330,1860956,4.0,2003-06-10,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252331,165195,3.0,2005-09-18,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252332,7940,3.0,2004-01-13,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252333,2183302,2.0,2001-10-20,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252334,533712,5.0,2004-09-14,4388,NaN,Ancient Civilizations: Rome and Pompeii
1252335,884603,1.0,2005-08-11,4388,NaN,Ancient Civilizations: Rome and Pompeii


In [7]:
# did a quick search. year should be 2001
combined['Year'].fillna(2001.0, inplace=True)

#### need to check on this

In [8]:
f = ['count','mean']

df_movie_summary = combined.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.8),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = combined.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.8),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

Movie minimum times of review: 209.0
Customer minimum times of review: 81.0


In [9]:
"""reader = Reader()

data = Dataset.load_from_df(combined[['Cust_Id', 'Movie_Id', 'Rating']], reader)
data.split(n_folds=3)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])"""

"reader = Reader()\n\n# get just top 100K rows for faster run time\ndata = Dataset.load_from_df(combined[['Cust_Id', 'Movie_Id', 'Rating']], reader)\ndata.split(n_folds=3)\n\nsvd = SVD()\ncross_validate(svd, data, measures=['RMSE', 'MAE'])"

In [ ]:
# this rmse is more than 1...

In [10]:
reader = Reader()

data = Dataset.load_from_df(combined[['Cust_Id', 'Movie_Id', 'Rating']], reader)


# define a cross-validation iterator


kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.9263
RMSE: 0.9255
RMSE: 0.9238


In [ ]:
# jin - the score means what....? how to evaluate?????

In [11]:
movies = pd.read_csv('./datasets/movie_titles.csv', encoding='latin1', usecols = [0,1,2], names = ['Movie_Id','Year', 'Title']).set_index('Movie_Id')


In [12]:
movies.head()

,Year,Title
Movie_Id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW


### Collaborative filtering

In [14]:
df_822109 = combined[(combined['Cust_Id'] == 822109) & (combined['Rating'] == 5)]
df_822109 = df_822109.set_index('Movie_Id')
df_822109

,Cust_Id,Rating,Date,Year,Title
Movie_Id,,,,,
1,822109,5.0,2005-05-13,2003.0,Dinosaur Planet
571,822109,5.0,2005-04-25,1999.0,American Beauty
607,822109,5.0,2005-04-07,1994.0,Speed
985,822109,5.0,2005-04-25,1999.0,The Mummy
1144,822109,5.0,2005-05-21,1991.0,Fried Green Tomatoes
1220,822109,5.0,2005-04-25,2004.0,Man on Fire
1905,822109,5.0,2005-05-21,2003.0,Pirates of the Caribbean: The Curse of the Bla...
3029,822109,5.0,2005-04-07,1998.0,T-Rex: Back to the Cretaceous: IMAX
3106,822109,5.0,2005-04-25,1990.0,Ghost


### there are tvseries also. how? or ok? this is really slow also

In [17]:
# predict

user_822109 = movies.copy()
user_822109 = user_822109.reset_index()
user_822109 = user_822109[~user_822109['Movie_Id'].isin(drop_movie_list)]

# getting full dataset
data = Dataset.load_from_df(combined[['Cust_Id', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset()
algo.fit(trainset)

user_822109['Estimate_Score'] = user_822109['Movie_Id'].apply(lambda x: algo.predict(822109, x).est)

user_822109 = user_822109.drop('Movie_Id', axis = 1)

user_822109 = user_822109.sort_values('Estimate_Score', ascending=False)
user_822109.head(10)

,Year,Title,Estimate_Score
3961,2003.0,Finding Nemo (Widescreen),5.000000
4426,2001.0,The West Wing: Season 3,5.000000
3455,2004.0,Lost: Season 1,5.000000
3445,2002.0,Spirited Away,5.000000
4382,2004.0,Farscape: The Peacekeeper Wars,4.979029
3937,2004.0,Shrek 2,4.938737
2781,1995.0,Braveheart,4.929759
3667,2000.0,Farscape: Season 2,4.928813
885,2004.0,Ray,4.908817
4097,2002.0,Angel: Season 4,4.903813


In [ ]:
# create a subset as a test??
# how to test

In [ ]:
# use recall?

# other codes

In [ ]:
"""reader = Reader()

data = Dataset.load_from_df(combined[['Cust_Id', 'Movie_Id', 'Rating']], reader)


In [ ]:
"""# define a cross-validation iterator
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)"""

In [ ]:
"""def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations"""

In [ ]:
#predictions = recommend_movies(preds, 822109, movies, ratings, 20)
#predictions

In [ ]:
#evaluate(svd, data, measures=['RMSE', 'MAE'])

In [ ]:
#df_822109 = combined[(combined['Cust_Id'] == 822109) & (combined['Rating'] == 5)]
#df_822109 = df_822109.set_index('Movie_Id')
#print(df_822109)

In [ ]:
'''user_785314 = user_785314.reset_index()
user_785314 = user_785314[~user_785314['Movie_Id'].isin(drop_movie_list)]

# getting full dataset
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset()
svd.train(trainset)

user_785314['Estimate_Score'] = user_785314['Movie_Id'].apply(lambda x: svd.predict(785314, x).est)

user_785314 = user_785314.drop('Movie_Id', axis = 1)

user_785314 = user_785314.sort_values('Estimate_Score', ascending=False)
print(user_785314.head(10))'''

In [ ]:
"""from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

"""

In [ ]:
#top_n[0]

In [ ]:
# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
#   print(uid, [iid for (iid, _) in user_ratings])

In [19]:
"""
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir


def read_item_names():

# First, train the algortihm to compute the similarities between items
data = Dataset.load_from_df(combined[['Cust_Id', 'Movie_Id', 'Rating']], reader)
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo1 = KNNBaseline(sim_options=sim_options)
algo1.fit(trainset)

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_to_rid['Toy Story']
toy_story_inner_id = algo1.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo1.get_neighbors(toy_story_inner_id, k=10)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = (algo1.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

print()
print('The 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)"""

"\nfrom surprise import KNNBaseline\nfrom surprise import Dataset\nfrom surprise import get_dataset_dir\n\n\ndef read_item_names():\n\n# First, train the algortihm to compute the similarities between items\ndata = Dataset.load_from_df(combined[['Cust_Id', 'Movie_Id', 'Rating']], reader)\ntrainset = data.build_full_trainset()\nsim_options = {'name': 'pearson_baseline', 'user_based': False}\nalgo1 = KNNBaseline(sim_options=sim_options)\nalgo1.fit(trainset)\n\n# Read the mappings raw id <-> movie name\nrid_to_name, name_to_rid = read_item_names()\n\n# Retrieve inner id of the movie Toy Story\ntoy_story_raw_id = name_to_rid['Toy Story']\ntoy_story_inner_id = algo1.trainset.to_inner_iid(toy_story_raw_id)\n\n# Retrieve inner ids of the nearest neighbors of Toy Story.\ntoy_story_neighbors = algo1.get_neighbors(toy_story_inner_id, k=10)\n\n# Convert inner ids of the neighbors into names.\ntoy_story_neighbors = (algo1.trainset.to_raw_iid(inner_id)\n                       for inner_id in toy_sto